In [4]:
# Create docker container for agent environment

import docker

# Create a Docker client from the environment
client = docker.from_env()

# Pull the image (Ubuntu and Git pre-installed)
# Use `ubuntu` base and install git during container creation
container = client.containers.run(
    image="ubuntu:latest",              # Base image
    command="sleep infinity",           # Keep container running
    detach=True,
    tty=True,                           # Enables command execution like a shell
    stdin_open=True
)

# Install git inside the container
exit_code, output = container.exec_run("bash -c 'apt-get update && apt-get install -y git'")
if exit_code != 0:
    container.stop()
    container.remove()
    raise Exception(f"Failed to install git: {output}")

print(f"Successfully installed git on container: {container.id}")


In [1]:
# Define tools

def parse_command(cmd: str) -> bool:
    allowed_unix = ["ls", "cd", "pwd", "cat", "mkdir"]
    allowed_git = ["add", "commit", "push", "pull", "status"]
    allowed_commands = allowed_unix + allowed_git
    
    if cmd.split()[0] not in allowed_commands:
        return False
    
    return True


def execute_command(cmd: str) -> str:
    exit_code, output = container.exec_run(cmd)
    if exit_code != 0:
        _, stderr = output
        return f"Error: {stderr.decode()}"
    return output.decode()


def execute_unix_command(cmd: str) -> str:
    return execute_command(f"bash -c '{cmd}'")

def execute_git_command(cmd: str) -> str:
    return execute_command(f"git {cmd}")

In [ ]:
# Create docker container for agent environment

